Documentation

https://praw.readthedocs.io/en/stable/getting_started/authentication.html 

In [1]:
import praw

user_agent = "Reddit_Scrapper 1.0 by u/Ansamzi"
reddit = praw.Reddit(
    client_id="GxxMVD29d1RBDXIqdjDnjA",
    client_secret="zTpzmgGGuSCnL2BlzptPX5yEBC_CCw",
    redirect_uri="http://localhost:8080",
    user_agent=user_agent
    )

In [2]:
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta

In [3]:
start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 6, 30)

# Calculate the difference in days
days_difference = (end_date - start_date).days

In [4]:
start_date = datetime.utcnow() - timedelta(days=days_difference)

In [5]:
headlines = set()
for submission in reddit.subreddit("worldnews").hot(limit=10):
 print(submission.created_utc)
 print(submission.title)#Subreddit Title
 print(submission.id) #ID
 print(submission.author) #Author of the subreddit
 print(submission.created_utc) #Date and time being created
 print(submission.score) # Average Score
 print(submission.upvote_ratio) # Upvote ratio
 print(submission.url) # Like to the Subreddit
 break

print(len(headlines))

1700366537.0
/r/WorldNews Live Thread: Russian Invasion of Ukraine Day 634, Part 1 (Thread #780)
17yoiuz
WorldNewsMods
1700366537.0
966
0.96
https://www.reddit.com/live/18hnzysb1elcs
0


In [6]:
headlines = set()
for submission in reddit.subreddit("worldnews").hot(limit=None):
 headlines.add(submission.title)
 headlines.add(submission.created_utc)
print(len(headlines))

1208


In [7]:
#Create a dataframe from the scrapped data 
news_df = pd.DataFrame(headlines)
news_df.head()

,0
0,1700141056.0
1,Suella Braverman's releases scathing letter ta...
2,1700159491.0
3,1700139012.0
4,1700173829.0


In [8]:
datetime_object = datetime.utcfromtimestamp(news_df.iloc[1205, 0])
datetime_object.strftime('%Y-%m-%d %H:%M:%S UTC')

'2023-11-16 21:56:10 UTC'

In [9]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1208 entries, 0 to 1207
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       1208 non-null   object
dtypes: object(1)
memory usage: 9.6+ KB


In [10]:
headlines = set()
for submission in reddit.subreddit("worldnews").new(limit=None):
 if submission.created_utc > start_date.timestamp():
    headlines.add(submission.title)
print(len(headlines))

545


In [11]:
# Get news from the specified day
start_date = datetime(2023, 11, 19, 0, 0, 0)
end_date = datetime(2023, 11, 19, 23, 59, 59)

# Convert start and end times to Unix timestamp
start_timestamp = start_date.timestamp()
end_timestamp = end_date.timestamp()

headlines = set()
for submission in reddit.subreddit("worldnews").hot(limit=10):
   if start_timestamp <= submission.created_utc <= end_timestamp:
       headlines.add(submission.title)

In [12]:
headlines

{"'I'm being cut off from my family': Finland's Russian community hits out against border closures. At a protest in the capital, people raised concerns about a new 'Iron Curtain'.",
 '/r/WorldNews Live Thread: Russian Invasion of Ukraine Day 634, Part 1 (Thread #780)',
 '31 premature babies safely evacuated from Shifa Hospital to southern Gaza, WHO says',
 'Biden warns U.S. could sanction Israeli settlers who attack Palestinians',
 "By supporting Ukraine, we're keeping American troops out of the war – US President",
 'EU strikes deal on new ‘ecocide’ rules to put polluters in jail',
 'Russia moves to ban ‘international LGBT public movement’ for ‘extremism’',
 'Russian general critical of army found dead, along with his wife',
 'Saudi Media reports Houthi rebels take Israeli cargo ship'}

In [13]:
# Define the start and end times for the day
start_date = datetime(2023, 10, 19, 0, 0, 0)
end_date = datetime(2023, 10, 19, 23, 59, 59)

start_timestamp = start_date.timestamp()
end_timestamp = end_date.timestamp()

headlines = set()
for submission in reddit.subreddit("worldnews").new(limit=10):
   if start_timestamp <= submission.created_utc <= end_timestamp:
       headlines.add(submission.title)

headlines

set()

Seems like I can't do it for certain days

trying this method

https://github.com/dmarx/psaw

didn't work :(

In [18]:
subreddit = reddit.subreddit("worldnews")

posts = subreddit.top(time_filter = "all", limit = None)
 
posts_dict = {"Title": [], "Post Text": []}

start_date = datetime(2023, 10, 19, 0, 0, 0).timestamp()
end_date = datetime(2023, 10, 19, 23, 59, 59).timestamp()

for post in posts:
    date = post.created_utc
    if date > start_date and date < end_date:
        posts_dict["Title"].append(post.title)
        posts_dict["Post Text"].append(post.selftext)
        
all_posts = pd.DataFrame(posts_dict)
all_posts

,Title,Post Text


In [ ]:
import requests
from datetime import datetime

def get_top_news(api_key, date):
    # Format the date
    formatted_date = date.strftime("%Y-%m-%d")

    url = f'https://newsapi.org/v2/everything?q=&from={formatted_date}&to={formatted_date}&apiKey={api_key}'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        articles = data.get('articles', [])
        return [article['title'] for article in articles]
    else:
        print(f"Error {response.status_code}: {response.text}")
        return None

if __name__ == "__main__":
    # Replace 'YOUR_API_KEY' with your actual News API key
    api_key = '25b33ca710ad42968acc183a39eb7567'
    
    # Replace '2022-01-01' with the date you're interested in
    date_to_fetch = datetime.strptime('2022-01-01', '%Y-%m-%d')

    news_titles = get_top_news(api_key, date_to_fetch)

    if news_titles:
        print(f"Top news titles for {date_to_fetch.strftime('%Y-%m-%d')}:\n")
        for i, title in enumerate(news_titles, 1):
            print(f"{i}. {title}")
    else:
        print("Failed to fetch news titles.")
